# Fichier: LECTURE ET ANALYSE DES CLUSTERS EN STOCK
### Importation des librairies

In [ ]:
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.offline.offline import _plot_html
import plotly.graph_objs as go
print (__version__)
init_notebook_mode(connected=True)

from mpl_toolkits import mplot3d
import matplotlib.pylab as plt
from matplotlib.pylab import rcParams
%matplotlib notebook
#%matplotlib inline

from tslearn.clustering import silhouette_score

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
init_notebook_mode(connected=True)

from utils import series_supp as ss
from utils import data_factory as df
from utils import k_mean as km
from utils import k_shape as ks
from utils.statics_func import *
from utils import plot

### Initialise la factory et créé les instances de Support de Series: ss.SeriesSupp

In [ ]:
cwd = os.getcwd()
os.chdir(cwd)
print(cwd)
factory = df.DataFactory(cwd)
global_ploter = plot.Plot(None)

RG24 = ss.SeriesSupp(cwd, factory, "RG24")
RG1 = ss.SeriesSupp(cwd, factory, "RG1")
GW = ss.SeriesSupp(cwd, factory, "GW")

### Importation des datasets depuis les fichiers de stockage csv

In [ ]:
GW.import_dataset()
RG24.import_dataset()

### Création des instances de Clustering
Commenter ou décommenter pour choisir le cluster_GW voulu selon Kshape ou Kmean
* GW: Grand Water -> les piezomètres
* RG: Rain gauge -> les pluviomètres

In [ ]:
#cluster_GW = km.Kmean(GW)
cluster_GW = ks.Kshape(GW)
cluster_RG = km.Kmean(RG24)

## Importation du fichier Pickles contenant le clustering voulu

In [ ]:
name_file = openfile_dialog()
cluster_GW.read_cluster(name_file[0])

In [ ]:
#name_file = "Kshape_GW_[2011, 2012, 2013, 2014, 2015][1, 2, 3, 4, 5, 12]_True_5"
#cluster_GW.read_cluster("cluster\\29_06\\" + name_file + ".pkl")

## Affichage des cluster
* Prototypes en rouge
* informations en dessous
* repartition des objets dans les cluster

In [ ]:
cluster_GW.ploter.plot_cluster_light()

## Affichage d'une multitude d'informations sur le clustering observé

In [ ]:
cluster_GW.show_info()
#print("rounded values: " + str(cluster_GW.ss.rounded))
print("Repartition des objets au sein des clusters")
cluster_GW.cluster_counter()
cluster_GW.counter
print("Nombre total d'objets: " + str(len(cluster_GW.ts_clust)))
print("Repartition des objets dans les cluster selon leur capteur, en rouge le cluster le plus représentatif du capteur")
cluster_GW.capteur_parser()
cluster_GW.style_df("max", cluster_GW.get_captor_distribution_in_cluster())

In [ ]:
cluster_GW.ploter.change_mode(3)
#cluster_GW.cluster_by_fullname

### Silhouette score cluster quality indice
could be huge and long computation due to massive dataset

In [ ]:
#sil = silhouette_score(cluster_GW.ts, cluster_GW.ts_clust, metric="softdtw", metric_params = {"gamma_sdtw": .01})
#print("Silhouette score: " + str(sil))

## Analyse: Cluster

### n_obs: numéro du cluster analysé
* Veuillez entrer le numéro du cluster que vous souhaitez analyser plus en détaille dans la variable **n_obs**

In [ ]:
n_obs = 1

* Affichage 3D de la répartition géographique des ouvrages 

In [ ]:
cluster_GW.geo.plotly_3D(cluster_GW.cluster_by_name[n_obs])

* Histogramme de la répartition des objets dans ce cluster selon de quel capteur ils sont issus et de quelle partie de l'année (mois ou semaine selon granularité)

In [ ]:
cluster_GW.ploter.plot_histo_2(n_obs)

* Affichage de tous les objets présent dans le cluster sélectionné 

In [ ]:
cluster_GW.clust_hoverview(n_obs)

## Setup GW selon clusters

In [ ]:
print(cluster_GW.ss.tmp_dataset == cluster_GW.ss.dataset)

In [ ]:
cluster_GW.ss.reset_dataset()

In [ ]:
# Smoothing
cluster_GW.ss.dict_smooth()

In [ ]:
# Les decoupages
cluster_GW.ss.split_data_years()
if cluster_GW.ss.days and len(cluster_GW.months) == 12:
    cluster_GW.ss.split_data_weeks()
elif cluster_GW.ss.days and len(cluster_GW.months) != 12:
    cluster_GW.ss.split_data_months()
    cluster_GW.ss.split_data_weeks()
elif not cluster_GW.ss.days:
    cluster_GW.ss.split_data_months()

In [ ]:
# Round 0.1
# Normalisation mean = 0 standard deviation = 1
#cluster_GW.ss.dict_norm()

# Analyse un capteur en particulier: 
### Veuillez entrer le nom du capteur souhaite

In [ ]:
cap = "GW_239"

In [ ]:
clts = cluster_GW.get_clust_part_for_captor(cap)

### Traitement et initialisation des vues

* Creation de la matrice de distance et récupération du Pluviomètre le plus proche de notre piezometre

In [ ]:
dm = cluster_GW.geo.distance_matrix()
cluster_GW.geo.distance_dict()
RG_captor = cluster_GW.geo.get_minrange_rg(clts[0])

* Nom du capteur pluviometrique

In [ ]:
RG_captor

* Traitement des données pluviometrique pour s'adapter au données souterraines, modification manuelle de RG24.years

In [ ]:
RG24.reset_dataset()
RG24.years = list(range(2015, 2016))

* En commentaire l'adaptation automatique des années, mais par soucis de clarté il est souvent préférable de choisir les années de couvertures
* Normalisation des données de pluie possible en decommentant la dernière ligne **#RG24.dict_norm()**

In [ ]:
#RG24.years = cluster_GW.ss.years
RG24.months = cluster_GW.ss.months

RG24.split_data_years()
if cluster_GW.ss.days:
    RG24.split_data_weeks()
else:
    RG24.split_data_months()
#RG24.dict_norm()

* Recuperation des données liées au capteur pluviométrique le plus proche

In [ ]:
RG_captor_values = RG24.get_data_from_captor(str("24h_" + RG_captor))

Construction et affichage des graphiques:
* Les données piezometriques avec la colorimetrie en legende (au dessus) les objet en noir sont ceux qui n'ont pas été sélectionnés pour la classification, dû à une mauvaise taille par manque d'information ou ne correspondant pas au format horaire.
* Les données de pluie sur la plage annuelle et le découpage préféré en amont.
* Ces mêmes données vu de manière cumulative

In [ ]:
global_ploter.change_mode(3)
cluster_GW.aff_color()
global_ploter.plot_scatter_by_capteur_color_cluster(data = cluster_GW.ss.tmp_dataset, dict_clust = clts)
global_ploter.plot_scatter(RG_captor_values)

RG24.reset_dataset()
for k, v in RG24.tmp_dataset.items():
    if RG_captor in k:
        v["Valeur"] = v.Valeur.cumsum()        
RG_captor_values = RG24.get_data_from_captor(str("24h_" + RG_captor))
global_ploter.plot_scatter(RG_captor_values)